In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
from pathlib import Path
import numpy as np
import multiprocessing

In [2]:
# check if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Model

In [3]:
v = 3     # model version
in_c = 2  # number of input channels
num_c = 1 # number of classes to predict

In [4]:
# The optical flow input will look like this
# of = torch.randn(1,2,640,480)

In [5]:
model = EfficientNet.from_pretrained(f'efficientnet-b{v}', in_channels=in_c, num_classes=num_c)
model.to(device);

Loaded pretrained weights for efficientnet-b3


#### The output of the model will look like this

In [6]:
# of = of.to(device)
# model(of).item()

## Data

In [7]:
# directory with the optical flow images
of_dir = '../opical-flow-estimation-with-RAFT/output'
# labels as txt file
labels_f = 'train.txt'

In [8]:
class OFDataset(Dataset):
    def __init__(self, of_dir, label_f):
        self.len = len(list(Path(of_dir).glob('*.npy')))
        self.of_dir = of_dir
        self.label_file = open(label_f).readlines()
    def __len__(self): return self.len
    def __getitem__(self, idx):
        of_array = np.load(Path(self.of_dir)/f'{idx}.npy')
        of_tensor = torch.squeeze(torch.Tensor(of_array))
        label = float(self.label_file[idx].split()[0])
        return [of_tensor, label]

In [9]:
ds = OFDataset(of_dir, labels_f)

In [10]:
# 80% of data for training
# 20% of data for validation
train_split = .8

In [11]:
ds_size = len(ds)
indices = list(range(ds_size))
split = int(np.floor(train_split * ds_size))
train_idx, val_idx = indices[:split], indices[split:]

In [12]:
sample = ds[3]
assert type(sample[0]) == torch.Tensor
assert type(sample[1]) == float

In [13]:
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [14]:
cpu_cores = multiprocessing.cpu_count()
cpu_cores

16

In [15]:
train_dl = DataLoader(ds, batch_size=4, sampler=train_sampler, num_workers=0)
val_dl = DataLoader(ds, batch_size=4, sampler=val_sampler, num_workers=0)

## Train

In [16]:
epochs = 32
log_train_steps = 100

In [17]:
criterion = nn.MSELoss()
opt = optim.Adam(model.parameters())

In [18]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, sample in enumerate(train_dl):
        of_tensor = sample[0].cuda()
        label = sample[1].float().cuda()
        opt.zero_grad()
        pred = torch.squeeze(model(of_tensor))
        loss = criterion(pred, label)
        loss.backward()
        opt.step()
    # validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for j, val_sample in enumerate(val_dl):
            of_tensor = val_sample[0].cuda()
            label = val_sample[1].float().cuda()
            pred = torch.squeeze(model(of_tensor))
            loss = criterion(pred, label)
            val_losses.append(loss)
        print(f'{epoch}: {sum(val_losses)/len(val_losses)}')

0: 3.8471200466156006
1: 2.4729902744293213
2: 2.4499619007110596
3: 5.279375076293945
4: 3.8110218048095703
5: 2.7851922512054443
6: 2.4250690937042236
7: 3.3132076263427734
8: 3.619356155395508
9: 1.777315616607666
10: 1.8335007429122925
11: 9.120901107788086
12: 2.0165297985076904
13: 3.6629514694213867
14: 8.55787467956543
15: 2.6016652584075928
16: 2.4927313327789307
17: 2.273057699203491
18: 4.339877605438232
19: 3.094835042953491
20: 68.25261688232422
21: 3.1047558784484863
22: 4.118056297302246
23: 3.3090903759002686
24: 3.118864059448242
25: 2.825462579727173
26: 3.055203676223755
27: 2.5582165718078613
28: 2.919032573699951
29: 2.65401291847229
30: 3.4333128929138184
31: 4.077401161193848


In [19]:
torch.save(model.state_dict(), 'model/b3_32.pth')